In [35]:
import pandas as pd

from autoqchem.molecule import molecule
from autoqchem.pbs_manager import pbs_manager
from autoqchem.helper_classes import pbs_status
from autoqchem.draw_utils import draw


## pbs_manager
### autoQchem job을 관리하는 manager. 누리온 서버에 접속하여 job 생성 및 upload, 삭제 등 job을 관리한다

In [2]:
pm=pbs_manager(user='a1793a01', host='nurion.ksc.re.kr', sftp_host='nurion-dm.ksc.re.kr')

In [45]:
## OTP가 필요. password = nuri2024@@
pm.connect()

Password(OTP):  ········
Password:  ········


In [5]:
## Set molecule information
my_smiles=[
    "C(C1C(C(C(C(O1)O)O)O)O)O",
    "c1ccc(F)cc1",
]

In [6]:
mols = []
for smile in my_smiles:
    mols.append(molecule(smile, num_conf=10))

# Gaussian Job setup
### Gaussian Job 생성 조건

In [7]:
for mol in mols:
    pm.create_jobs_for_molecule(mol, light_basis_set='6-31G**', solvent='Ethanol', wall_time='06:00:00')

In [36]:
pm.get_job_stats(split_by_can=True)

status,done,submitted
can,,
C(C1C(C(C(C(O1)O)O)O)O)O,0,8
c1ccc(F)cc1,1,0


In [32]:
pm.connect_sftp()

Password(OTP):  ········
Password:  ········


In [11]:
## job status가 create 인 job을 run
pm.submit_jobs()

In [46]:
pm.connection.run(f"id {pm.user}")

uid=404162301(a1793a01) gid=4041623(pd1623) groups=4041623(pd1623),1000009(gauss)


<Result cmd='id a1793a01' exited=0>

In [47]:
out = pm.connection.run(f"qstat -u {pm.user}", pty=False, timeout=5)
out

<Result cmd='qstat -u a1793a01' exited=0>

In [48]:
pm.retrieve_jobs()

There are 0 running/pending jobs, 8 finished jobs.


In [49]:
pm.get_job_stats(split_by_can=True)

status,done
can,
C(C1C(C(C(C(O1)O)O)O)O)O,8
c1ccc(F)cc1,1


In [50]:
pm.upload_done_molecules_to_db(tags=["yh_test"])

In [3]:
from autoqchem.db_local_functions import db_connect

db = db_connect()
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'autoqchem')

In [5]:
db.list_collection_names()

['qchem_descriptors', 'log_files', 'molecules', 'tags']